In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## 1. Загрузка данных в переменную

In [ ]:
data = pd.read_csv('/content/exams.csv')

In [ ]:
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


## Описание и постановка задачи.

Исходя из представленых колонок, можно сделать вывод о том, что идеальной задачей для такого датасета является регрессия. В таком случае целевой переменной следует выбрать math score.

In [ ]:
data.describe()

,math score,reading score,writing score
count,1000.000000,1000.000000,1000.000000
mean,66.396000,69.002000,67.738000
std,15.402871,14.737272,15.600985
min,13.000000,27.000000,23.000000
25%,56.000000,60.000000,58.000000
50%,66.500000,70.000000,68.000000
75%,77.000000,79.000000,79.000000
max,100.000000,100.000000,100.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


## 3. Предобработка данных

### 3.1 Проверим пропуски данных

In [ ]:
missing_values = data.isnull().sum()
missing_values1 = data.isna().sum()
print("Количество пропусков в каждой колонке: ")
print(missing_values, "\n")
print("#######################################", "\n")
print(missing_values1)

Количество пропусков в каждой колонке: 
gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64 

####################################### 

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64


In [ ]:
data.applymap(lambda x: x is None)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False
996,False,False,False,False,False,False,False,False
997,False,False,False,False,False,False,False,False
998,False,False,False,False,False,False,False,False


оценим целевую

In [ ]:
data['math score'].value_counts()

math score
63    34
71    30
77    30
74    28
57    27
      ..
26     2
23     1
29     1
34     1
25     1
Name: count, Length: 77, dtype: int64

## 3.3 Поделим выборки

In [ ]:
X = data.drop('math score', axis=1)
y = data['math score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Размер тренировочной выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)

Размер тренировочной выборки: (700, 7)
Размер тестовой выборки: (300, 7)


### 3.2 Отнормируем численные переменные, закодируем качественные

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
numeric_preds = ['reading score', 'writing score']
categorial_preds = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_preds),
        ('cat', categorical_transformer, categorial_preds)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Размер предобработанной тренировочной выборки:", X_train_processed.shape)
print("Размер предобработанной тестовой выборки:", X_test_processed.shape)

Размер предобработанной тренировочной выборки: (700, 19)
Размер предобработанной тестовой выборки: (300, 19)


In [ ]:
scaler = StandardScaler()

y_train_preprocessed = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_preprocessed = scaler.transform(y_test.values.reshape(-1, 1))

## Обучим на тренировочном множестве

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

### Линейная модель

In [ ]:
Linear_regression = LinearRegression()
Linear_regression.fit(X_train_processed, y_train_preprocessed)

y_pred_linear_regression = Linear_regression.predict(X_test_processed)

### Деревянная модель

In [ ]:
decision_tree_regressor = DecisionTreeRegressor()
decision_tree_regressor.fit(X_train_processed, y_train_preprocessed)

y_pred_decision_tree_regressor = decision_tree_regressor.predict(X_test_processed)

### K-ближайших соседей

In [ ]:
k_neighbors_regressor = KNeighborsRegressor()
k_neighbors_regressor.fit(X_train_processed, y_train_preprocessed)

y_pred_k_neighbors_regressor = k_neighbors_regressor.predict(X_test_processed)

### Случайный лес

In [ ]:
random_forest_regressor = RandomForestRegressor()
random_forest_regressor.fit(X_train_processed, y_train_preprocessed)

y_pred_random_forest_regressor = random_forest_regressor.predict(X_test_processed)

<ipython-input-73-381a1ed2a5c1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_regressor.fit(X_train_processed, y_train_preprocessed)


## Посчитаем метрики!

In [ ]:
''' Линейная модель '''

### MAE ###
mae_linear_regression = mean_absolute_error(y_test_preprocessed, y_pred_linear_regression)
print("Средняя абсолютная ошибка (MAE) линейной регрессии:", mae_linear_regression)

### RMSE ###
mse_linear_regression = mean_squared_error(y_pred_linear_regression, y_test_preprocessed)
rmse_linear_regression = np.sqrt(mse_linear_regression)
print("Среднеквадратическая ошибка (RMSE) линейной регрессии:", rmse_linear_regression)

### MAPE ###
mape_linear_regression = mean_absolute_percentage_error(y_test_preprocessed, y_pred_linear_regression)
print("Средняя абсолютная процентная ошибка (MAPE) линейной регрессии:", mape_linear_regression)

Средняя абсолютная ошибка (MAE) линейной регрессии: 0.29227452826606315
Root Mean Squared Error (RMSE) of Linear Regression: 0.35534948818425305
Mean Absolute Percentage Error (MAPE) of Linear Regression: 9.011732875156342


In [ ]:
''' Деревянная модель '''

### MAE ###
mae_decision_tree_regressor = mean_absolute_error(y_test_preprocessed, y_pred_decision_tree_regressor)
print("Средняя абсолютная ошибка (MAE) дерева решений:", mae_decision_tree_regressor)

### RMSE ###
mse_decision_tree_regressor = mean_squared_error(y_test_preprocessed, y_pred_decision_tree_regressor)
rmse_decision_tree_regressor = np.sqrt(mse_decision_tree_regressor)
print("Среднеквадратическая ошибка (RMSE) дерева решений:", rmse_decision_tree_regressor)

### MAPE ###
mape_decision_tree_regressor = mean_absolute_percentage_error(y_test_preprocessed, y_pred_decision_tree_regressor)
print("Средняя абсолютная процентная ошибка (MAPE) дерева решений:", mape_decision_tree_regressor)

Средняя абсолютная ошибка (MAE) дерева решений: 0.4307885432655875
Среднеквадратическая ошибка (RMSE) дерева решений: 0.5390944763294991
Средняя абсолютная процентная ошибка (MAPE) дерева решений: 12.747038129793895


In [ ]:
''' К-ближайших '''

### MAE ###
mae_k_neighbors_regressor = mean_absolute_error(y_test_preprocessed, y_pred_k_neighbors_regressor)
print("Средняя абсолютная ошибка (MAE) KNeighbours:", mae_k_neighbors_regressor)

### RMSE ###
mse_k_neighbors_regressor = mean_squared_error(y_test_preprocessed, y_pred_k_neighbors_regressor)
rmse_k_neighbors_regressor = np.sqrt(mse_k_neighbors_regressor)
print("Среднеквадратическая ошибка (RMSE) KNeighbours:", rmse_k_neighbors_regressor)

### MAPE ###
mape_k_neighbors_regressor = mean_absolute_percentage_error(y_test_preprocessed, y_pred_k_neighbors_regressor)
print("Средняя абсолютная процентная ошибка (MAPE) KNeighbours:", mape_k_neighbors_regressor)


Средняя абсолютная ошибка (MAE) KNeighbours: 0.35124843417984164
Среднеквадратическая ошибка (RMSE) KNeighbours: 0.43250179668338834
Средняя абсолютная процентная ошибка (MAPE) KNeighbours: 14.748655374760714


In [ ]:
''' Случайный лес '''

### MAE ###
mae_random_forest_regressor = mean_absolute_error(y_test_preprocessed, y_pred_random_forest_regressor)
print("Средняя абсолютная ошибка (MAE) случайного леса:", mae_random_forest_regressor)

### RMSE ###
mse_random_forest_regressor = mean_squared_error(y_test_preprocessed, y_pred_random_forest_regressor)
rmse_random_forest_regressor = np.sqrt(mse_random_forest_regressor)
print("Среднеквадратическая ошибка (RMSE) случайного леса:", rmse_random_forest_regressor)

### MAPE ###
mape_random_forest_regressor = mean_absolute_percentage_error(y_test_preprocessed, y_pred_random_forest_regressor)
print("Средняя абсолютная процентная ошибка (MAPE) случайного леса:", mape_random_forest_regressor)

Средняя абсолютная ошибка (MAE) случайного леса: 0.32399420129493195
Среднеквадратическая ошибка (RMSE) случайного леса: 0.3921742809897942
Средняя абсолютная процентная ошибка (MAPE) случайного леса: 9.654172855572478
